Imports

In [1]:
from nb201 import NB201Benchmark
import numpy as np
from warmstart.utils_templates import FullTemplate
import ConfigSpace as CS
from ConfigSpace import Configuration
import ollama
import openai
from openai import AsyncOpenAI
from openai import OpenAI
import torchvision
from exp_baselines.bayesmark.data import ProblemType
import ast
from llambo.llambo import LLAMBO
from utils import convert_LLAMBO_df_to_synetune_dict
from utils import convert_synetune_dict_to_LLAMBO_compatible_format

from syne_tune_local.experiments.benchmark_definitions.nas201 import nas201_benchmark
from syne_tune_local.blackbox_repository import BlackboxRepositoryBackend
from syne_tune_local.backend.simulator_backend.simulator_callback import SimulatorCallback
from syne_tune_local import Tuner, StoppingCriterion

from typing import Optional, Dict, Any, List, Union
import logging
from syne_tune_local.optimizer.schedulers import FIFOScheduler
from syne_tune_local.optimizer.schedulers.searchers import StochasticAndFilterDuplicatesSearcher

openai.api_type = 'open_ai'
openai.api_version = '2020-11-07'
openai.api_base = 'https://api.openai.com/'
openai.api_key = 'sk-proj-cTiFSkfqsowfmg6FId0iT3BlbkFJfDi43ms9k13R9bKElYE1'

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/e.lushtaku/Library/Application Support/sagemaker/config.yaml
Dependencies of YAHPO are not imported since dependencies are missing. You can install them with
   pip install 'syne-tune[yahpo]'
or (for everything)
   pip install 'syne-tune[extra]'
Dependencies of YAHPO are not imported since dependencies are missing. You can install them with
   pip install 'syne-tune[yahpo]'
or (for everything)
   pip install 'syne-tune[extra]'


Load NB201 Benchmark

In [2]:
b = NB201Benchmark(path="./nb201.pkl", dataset='cifar10')
cs = b.get_configuration_space()
config = cs.sample_configuration()  # samples a configuration uniformly at random

print(cs)
print("Numpy representation: ", config.get_array())
print("Dict representation: ", config.get_dictionary())

#configuration from a dict
new_config = Configuration(cs, values=config.get_dictionary())
print(new_config)

y, cost = b.objective_function(config)
print("Test error: %f %%" % y)
print("Runtime %f s" % cost)

Configuration space object:
  Hyperparameters:
    op_0_to_1, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_0_to_2, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_0_to_3, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_1_to_2, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_1_to_3, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none
    op_2_to_3, Type: Categorical, Choices: {none, skip_connect, avg_pool_3x3, nor_conv_1x1, nor_conv_3x3}, Default: none

Numpy representation:  [0. 0. 3. 0. 0. 4.]
Dict representation:  {'op_0_to_1': 'none', 'op_0_to_2': 'none', 'op_0_to_3': 'nor_conv_1x1', 'op_1_to_2': 'none', 'op_1_to_3': 'none', 'op_2_to_3': 'nor_conv_3x3'}
Configuration(valu

/var/folders/b0/nhhvw1x54fggv0t8nzr9s4xm0000gp/T/ipykernel_84265/2721149981.py:7: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  print("Dict representation: ", config.get_dictionary())
/var/folders/b0/nhhvw1x54fggv0t8nzr9s4xm0000gp/T/ipykernel_84265/2721149981.py:10: DeprecationWarning: `Configuration` act's like a dictionary. Please use `dict(config)` instead of `get_dictionary` if you explicitly need a `dict`
  new_config = Configuration(cs, values=config.get_dictionary())


Arguments for LLAMBO

In [3]:
task_context = {
    'model': 'CNN',
    'task': 'classification',
    'metric': 'validation_loss',
    'num_samples': 50000,
    'image_size': 'height 32, width 32, and 3 channels',
    'num_feat': 32 * 32 * 3,
    'tot_feats': 32 * 32 * 3,
    'cat_feats': 0,
    'n_classes': 10,
    'lower_is_better': True,
    'hyperparameter_constraints': {
        'op_0_to_1': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        # [type, transform, [min_value, max_value]]
        'op_0_to_2': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_0_to_3': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_1_to_2': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_1_to_3': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]],
        'op_2_to_3': ['categorical', None, ["none", "skip_connect", "avg_pool_3x3", "nor_conv_1x1", "nor_conv_3x3"]]
    }
}

def eval_point(config):
    new_config = Configuration(b.get_configuration_space(), values=config)
    res = b.objective_function(new_config)
    res_dict = {
        "score": res[0],
        "train_time": res[1]
    }
    return config, res_dict

Ollama

In [4]:
# chat_engine = "llama3"
# model = ollama.pull(chat_engine)
# response = ollama.chat(model="llama3", messages=[{'role': 'user', 'content': 'Why is the sky blue?'}])
# print(response)
# ollama.list()

In [5]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True)


def fetch_statistics(dict, dataset):
    class_names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
    images = dataset.data
    labels = dataset.targets

    images_np = np.array(images)
    labels_np = np.array(labels)

    pixel_mean = np.mean(images_np / 255.)
    pixel_std = np.std(images_np / 255.)

    class_counts = np.bincount(labels_np)
    class_distribution = class_counts / len(labels_np)

    # Constructing the descriptive string for class distribution with class names
    class_distribution_str = ", ".join(
        f"{distribution * 100:.2f}% of datapoints belong to class {labels_np[i]}: {class_names[i]}"
        for i, distribution in enumerate(class_distribution) if i < len(labels_np)
    )
    print('class_distribution_str', class_distribution_str)
    dict['pixel_mean'] = pixel_mean
    dict['pixel_std'] = pixel_std
    dict['class_distribution'] = class_distribution_str

    return dict


task_context = fetch_statistics(task_context, trainset)

Files already downloaded and verified
class_distribution_str 10.00% of datapoints belong to class 6: Airplane, 10.00% of datapoints belong to class 9: Automobile, 10.00% of datapoints belong to class 9: Bird, 10.00% of datapoints belong to class 4: Cat, 10.00% of datapoints belong to class 1: Deer, 10.00% of datapoints belong to class 1: Dog, 10.00% of datapoints belong to class 2: Frog, 10.00% of datapoints belong to class 7: Horse, 10.00% of datapoints belong to class 8: Ship, 10.00% of datapoints belong to class 3: Truck


Warmstart

In [6]:
context = "Full_Context"
chat_engine = "gpt-4o-mini"
# metric = "acc"
# NUM_SEEDS = 10
# problem_type = ProblemType.clf


def extract_configs_from_response(response):
    # content = response['message']['content']
    content = response.choices[0].message.content
    print('content', content)
    start = content.find("[")
    end = content.rfind("]") + 1
    list_str = content[start:end]
    configurations = ast.literal_eval(list_str)
    return configurations


def is_dict_valid_in_config_space(d, config_space):
    try:
        # Attempt to create a Configuration object with the given dictionary and config space
        config = CS.Configuration(config_space, values=d)
        return True
    except:
        # Return False if the dictionary is not valid
        return False
    # Function to check if all dictionaries in a list are valid in the given configuration space


def check_all_list(parsed_dicts, config_space):
    for idx, d in enumerate(parsed_dicts):
        if not is_dict_valid_in_config_space(d, config_space):
            return False
    return True


def obtain_all_list_valid(resp, config_space):
    if check_all_list(resp, config_space):
        return resp
    print("fail")


def generate_init_conf(n_samples, client):
    template_object = FullTemplate(context=context, provide_ranges=True)
    user_message = template_object.add_context(config_space=cs, num_recommendation=n_samples, task_dict=task_context)
    print(user_message)

    messages = []
    messages.append({"role": "system", "content": "You are an AI assistant that helps people find information."})
    messages.append({"role": "user", "content": user_message})

    client = OpenAI(
        organization='org-HIWwqsnxyBU3xMl6PkwdjxdN',
        project='proj_g30ZlMFGBNu4qsUctIhNI5B3',
        api_key='sk-proj-cTiFSkfqsowfmg6FId0iT3BlbkFJfDi43ms9k13R9bKElYE1'
    )
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=4000,
        top_p=0.95,
        n=max(5, 3),  # e.g. for 5 templates, get 2 generations per template
        timeout=100
    )
    # response = openai.ChatCompletion.create(
    #                     engine=chat_engine,
    #                     messages=message,
    #                     temperature=0.7,
    #                     max_tokens=8,
    #                     top_p=0.95,
    #                     n=max(5, 3),  # e.g. for 5 templates, get 2 generations per template
    #                     request_timeout=10
    #                 )

    configs = extract_configs_from_response(response)

    return obtain_all_list_valid(configs, cs)

#print(generate_init_conf(3))

Llambo

In [7]:
# llambo = LLAMBO(task_context, sm_mode='discriminative', n_candidates=10, n_templates=2, n_gens=10,
#                 alpha=0.1, n_initial_samples=5, n_trials=4,
#                 init_f=generate_init_conf,
#                 bbox_eval_f=eval_point,
#                 chat_engine="llama3")
# llambo.seed = 0
# 
# # run optimization
# # configs, fvals = llambo.optimize(test_metric="score")

Searcher

In [8]:

logger = logging.getLogger(__name__)

MAX_RETRIES = 100


class LlamboSearcher(StochasticAndFilterDuplicatesSearcher):

    def __init__(
            self,
            config_space: Dict[str, Any],
            metric: Union[List[str], str],
            points_to_evaluate: Optional[List[dict]] = None,
            **kwargs,
    ):
        super().__init__(
            config_space,
            metric=metric,
            points_to_evaluate=points_to_evaluate,
            **kwargs,
        )
        self.X = []
        self.y = []

    def configure_scheduler(self, scheduler):
        from syne_tune_local.optimizer.schedulers.scheduler_searcher import (
            TrialSchedulerWithSearcher,
        )

        assert isinstance(
            scheduler, TrialSchedulerWithSearcher
        ), "This searcher requires TrialSchedulerWithSearcher scheduler"
        super().configure_scheduler(scheduler)

    def _train_model(self, train_data: np.ndarray, train_targets: np.ndarray) -> bool:
        """
        :param train_data: Training input feature matrix X
        :param train_targets: Training targets y
        :return: Was training successful?
        """
        llambo._update_observations(train_data, train_targets)
        return True
    
    def get_state(self) -> Dict[str, Any]:
        return dict(
            super().get_state(),
        )

    def _restore_from_state(self, state: Dict[str, Any]):
        super()._restore_from_state(state)

    def get_config(self, **kwargs) -> Optional[Dict[str, Any]]:
        suggestion = self._next_initial_config()
        if suggestion is None:
            if self.y:
                if self._train_model(np.array(self.X), np.array(self.y)):
                    suggestion = convert_LLAMBO_df_to_synetune_dict(llambo.get_config())
            
        return suggestion
        
    def _update(self, trial_id: str, config: Dict[str, Any], result: Dict[str, Any]):
        self.X.append(convert_synetune_dict_to_LLAMBO_compatible_format(config))
        self.y.append(result)
        
    def clone_from_state(self, state: Dict[str, Any]):
        raise NotImplementedError


In [9]:

logger = logging.getLogger(__name__)

MAX_RETRIES = 100


class MultiFidelityLLamboSearcher(LlamboSearcher):

    def __init__(
            self,
            config_space: Dict[str, Any],
            metric: Union[List[str], str],
            points_to_evaluate: Optional[List[dict]] = None,
            resource_attr: Optional[str] = None,
            **kwargs,
    ):
        super().__init__(
            config_space,
            metric=metric,
            points_to_evaluate=points_to_evaluate,
            **kwargs,
        )
        self.resource_attr = resource_attr
        self.resource_levels = []

    def configure_scheduler(self, scheduler):
        from syne_tune_local.optimizer.schedulers.multi_fidelity import (
            MultiFidelitySchedulerMixin,
        )

        super().configure_scheduler(scheduler)
        assert isinstance(
            scheduler, MultiFidelitySchedulerMixin
        ), "This searcher requires MultiFidelitySchedulerMixin scheduler"
        self.resource_attr = scheduler.resource_attr
       
    def _train_model(self, train_data: np.ndarray, train_targets: np.ndarray) -> bool:
        highest_resource_level = self._highest_resource_model_can_fit()
        if highest_resource_level is None:
            return False
        else:
            indices = np.where(self.resource_levels == highest_resource_level)
            sub_data = train_data[indices]
            sub_targets = train_targets[indices]
        return super()._train_model(sub_data, sub_targets)
     
    def _highest_resource_model_can_fit(self) -> Optional[int]:
        # find the highest resource level we have at least one data points of the positive class
        min_data_points = 4
        unique_resource_levels, counts = np.unique(
            self.resource_levels, return_counts=True
        )
        idx = np.where(counts >= min_data_points)[0]

        if len(idx) == 0:
            return None

        # collect data on the highest resource level
        return unique_resource_levels[idx[-1]]
    
    def get_state(self) -> Dict[str, Any]:
        return dict(
            super().get_state(),
        )

    def _restore_from_state(self, state: Dict[str, Any]):
        super()._restore_from_state(state)

    def _update(self, trial_id: str, config: Dict, result: Dict):
        super()._update(trial_id=trial_id, config=config, result=result)
        resource_level = int(result[self.resource_attr])
        self.resource_levels.append(resource_level)


Simple searcher combined with LLAMBO

In [10]:
# from syne_tune_local.optimizer.schedulers.synchronous import SynchronousGeometricHyperbandScheduler
# 
# logging.getLogger().setLevel(logging.WARNING)
# 
# random_seed = 1
# nb201_random_seed = 0
# n_workers = 1
# dataset_name = "cifar10"
# benchmark = nas201_benchmark(dataset_name)
# 
# max_resource_attr = benchmark.max_resource_attr
# trial_backend = BlackboxRepositoryBackend(
#     blackbox_name=benchmark.blackbox_name,
#     elapsed_time_attr=benchmark.elapsed_time_attr,
#     max_resource_attr=max_resource_attr,
#     dataset=dataset_name,
#     seed=nb201_random_seed,
# )
# 
# blackbox = trial_backend.blackbox
# nas_configuration_space = blackbox.configuration_space_with_max_resource_attr(
#     max_resource_attr
# )
# 
# points_to_evaluate = llambo.initialize_configs(5)
# points_to_evaluate = convert_LLAMBO_df_to_synetune_dict(points_to_evaluate)
# scheduler = SynchronousGeometricHyperbandScheduler(
#     config_space=nas_configuration_space,
#     max_resource_attr=max_resource_attr,
#     mode=benchmark.mode,
#     metric=benchmark.metric,
#     random_seed=random_seed,
#     searcher=MultiFidelityLLamboSearcher,
#     resource_attr=blackbox.fidelity_name(),
#     points_to_evaluate=points_to_evaluate,
#     brackets=1,
#     max_resource_level=2,
# )
# 
# max_wallclock_time = 300 # seconds
# stop_criterion = StoppingCriterion(max_wallclock_time=max_wallclock_time)
# print_update_interval = 700
# results_update_interval = 300
# tuner = Tuner(
#     trial_backend=trial_backend,
#     scheduler=scheduler,
#     stop_criterion=stop_criterion,
#     n_workers=n_workers,
#     sleep_time=0,
#     results_update_interval=results_update_interval,
#     print_update_interval=print_update_interval,
#     callbacks=[SimulatorCallback()],
# )
# 
# tuner.run()

In [11]:
# import os
# import openai
# 
# openai.api_type = 'open_ai'
# openai.api_version = '2020-11-07'
# openai.api_base = 'https://api.openai.com/v1'
# 
# openai.api_key = 'sk-proj-cTiFSkfqsowfmg6FId0iT3BlbkFJfDi43ms9k13R9bKElYE1'
# 
# # response = openai.ChatCompletion.create(
# #   model="gpt-4o-mini",
# #   messages=[
# #     {"role": "system", "content": "You are a helpful assistant."},
# #     {"role": "user", "content": "Who won the world series in 2020?"},
# #     {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
# #     {"role": "user", "content": "Where was it played?"}
# #   ]
# # )
# # print(response)
# 
# client = AsyncOpenAI(
#     organization='org-HIWwqsnxyBU3xMl6PkwdjxdN',
#     project='proj_g30ZlMFGBNu4qsUctIhNI5B3',
#     api_key='sk-proj-cTiFSkfqsowfmg6FId0iT3BlbkFJfDi43ms9k13R9bKElYE1'
# )
# 
# response = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[{"role": "user", "content": "Say this is a test"}],
#     
# )
# 
# print(response.choices[0].message.content)

In [12]:
client = AsyncOpenAI(
    organization='org-HIWwqsnxyBU3xMl6PkwdjxdN',
    project='proj_g30ZlMFGBNu4qsUctIhNI5B3',
    api_key='sk-proj-cTiFSkfqsowfmg6FId0iT3BlbkFJfDi43ms9k13R9bKElYE1'
)

llambo = LLAMBO(task_context, sm_mode='discriminative', n_candidates=10, n_templates=2, n_gens=10,
                alpha=0.1, n_initial_samples=5, n_trials=4,
                init_f=generate_init_conf,
                bbox_eval_f=eval_point,
                chat_engine="gpt-4o-mini",
                client = client)
llambo.seed = 0

# run optimization
# configs, fvals = llambo.optimize(test_metric="score")

[Search settings]: 
	n_candidates: 10, n_templates: 2, n_gens: 10, 
	alpha: 0.1, n_initial_samples: 5, n_trials: 4, 
	using warping: False, ablation: None, shuffle_features: False
[Task]: 
	task type: classification, sm: discriminative, lower is better: True
Hyperparameter search space: 
{'op_0_to_1': ['categorical',
               None,
               ['none',
                'skip_connect',
                'avg_pool_3x3',
                'nor_conv_1x1',
                'nor_conv_3x3']],
 'op_0_to_2': ['categorical',
               None,
               ['none',
                'skip_connect',
                'avg_pool_3x3',
                'nor_conv_1x1',
                'nor_conv_3x3']],
 'op_0_to_3': ['categorical',
               None,
               ['none',
                'skip_connect',
                'avg_pool_3x3',
                'nor_conv_1x1',
                'nor_conv_3x3']],
 'op_1_to_2': ['categorical',
               None,
               ['none',
                'skip

In [14]:
from syne_tune_local.optimizer.schedulers.synchronous import SynchronousGeometricHyperbandScheduler

logging.getLogger().setLevel(logging.WARNING)

random_seed = 1
nb201_random_seed = 0
n_workers = 1
dataset_name = "cifar10"
benchmark = nas201_benchmark(dataset_name)

max_resource_attr = benchmark.max_resource_attr
trial_backend = BlackboxRepositoryBackend(
    blackbox_name=benchmark.blackbox_name,
    elapsed_time_attr=benchmark.elapsed_time_attr,
    max_resource_attr=max_resource_attr,
    dataset=dataset_name,
    seed=nb201_random_seed,
)

blackbox = trial_backend.blackbox
nas_configuration_space = blackbox.configuration_space_with_max_resource_attr(
    max_resource_attr
)

points_to_evaluate = llambo.initialize_configs(5)
points_to_evaluate = convert_LLAMBO_df_to_synetune_dict(points_to_evaluate)
scheduler = SynchronousGeometricHyperbandScheduler(
    config_space=nas_configuration_space,
    max_resource_attr=max_resource_attr,
    mode=benchmark.mode,
    metric=benchmark.metric,
    random_seed=random_seed,
    searcher=MultiFidelityLLamboSearcher,
    resource_attr=blackbox.fidelity_name(),
    points_to_evaluate=points_to_evaluate,
    brackets=1,
    max_resource_level=2,
)

max_wallclock_time = 300 # seconds
stop_criterion = StoppingCriterion(max_wallclock_time=max_wallclock_time)
print_update_interval = 700
results_update_interval = 300
tuner = Tuner(
    trial_backend=trial_backend,
    scheduler=scheduler,
    stop_criterion=stop_criterion,
    n_workers=n_workers,
    sleep_time=0,
    results_update_interval=results_update_interval,
    print_update_interval=print_update_interval,
    callbacks=[SimulatorCallback()],
)

tuner.run()

You are assisting me with automated machine learning using CNN for a classification task. The classification performance is measured using validation_loss. The dataset has 50000 image samples with dimensions of height 32, width 32, and 3 channels. Class distribution is 10.00% of datapoints belong to class 6: Airplane, 10.00% of datapoints belong to class 9: Automobile, 10.00% of datapoints belong to class 9: Bird, 10.00% of datapoints belong to class 4: Cat, 10.00% of datapoints belong to class 1: Deer, 10.00% of datapoints belong to class 1: Dog, 10.00% of datapoints belong to class 2: Frog, 10.00% of datapoints belong to class 7: Horse, 10.00% of datapoints belong to class 8: Ship, 10.00% of datapoints belong to class 3: Truck. The pixel mean is 0.4733630004850874 and pixel standard deviation is 0.25156892506322026. I’m exploring a subset of hyperparameters detailed as: op_0_to_1 (string ('none', 'skip_connect', 'avg_pool_3x3', 'nor_conv_1x1', 'nor_conv_3x3')), op_0_to_2 (string ('no

ERROR:syne_tune_local.tuner:An error happened during the tuning, cleaning up resources and logging final resources before throwing the exception.


Adjusted alpha: 0.1 | [original alpha: 0.1], desired fval: 0.563490
--------------------
Resource summary (last result is reported):
 trial_id status  iter        hp_x0        hp_x1        hp_x2        hp_x3        hp_x4        hp_x5  epochs  metric_valid_error  metric_train_error  metric_runtime  metric_elapsed_time  metric_latency  metric_flops  metric_params  hp_epoch
        0 Paused     1 skip_connect nor_conv_3x3 avg_pool_3x3         none nor_conv_1x1 skip_connect       1              0.6776             0.62480       17.234987            17.234987        0.015232     47.104649       0.344346         1
        1 Paused     1 avg_pool_3x3         none nor_conv_1x1 skip_connect nor_conv_3x3 avg_pool_3x3       1              0.7451             0.65070       19.247335            19.247335        0.014326     47.104649       0.344346         1
        2 Paused     2 nor_conv_3x3 skip_connect         none avg_pool_3x3         none nor_conv_1x1       2              0.5074             0.4

ValueError: Unknown format code 'f' for object of type 'str'